## 1st step : load the document
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/MPdUH7bXpHR5muZztZfOQg.png" width="50%" alt="split"/>

In [1]:
filename = 'Universal Declaration of Human Righ.txt'


In [2]:
with open(filename, 'r') as file:
    # Read the contents of the file
    contents = file.read()
    print(contents)

Universal Declaration of Human Rights
The Universal Declaration of Human Rights (UDHR) is a milestone document in the history of human rights. Drafted by representatives with different legal and cultural backgrounds from all regions of the world, the Declaration was proclaimed by the United Nations General Assembly in Paris on 10 December 1948 (General Assembly resolution 217 A) as a common standard of achievements for all peoples and all nations. It sets out, for the first time, fundamental human rights to be universally protected and it has been translated into over 500 languages. The UDHR is widely recognized as having inspired, and paved the way for, the adoption of more than seventy human rights treaties, applied today on a permanent basis at global and regional levels (all containing references to it in their preambles). 

Preamble
Whereas recognition of the inherent dignity and of the equal and inalienable rights of all members of the human family is the foundation of freedom, j

## 2nd step : split the document into chunks 
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/0JFmAV5e_mejAXvCilgHWg.png" width="50%" alt="split"/>


In [3]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [4]:
loader = TextLoader(filename)
documents = loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,   # taille max de chaque chunk
    chunk_overlap=200, # recouvrement entre chunks (évite de perdre du contexte)
    separators=["\n\n", "\n", ".", " ", ""] # ordre de priorité pour couper
)
texts = text_splitter.split_documents(documents)
print("Nombre de chunks générés :", len(texts))
print("\nPremier chunk :\n")
print(texts[0].page_content[:500]) 

Nombre de chunks générés : 16

Premier chunk :

Universal Declaration of Human Rights
The Universal Declaration of Human Rights (UDHR) is a milestone document in the history of human rights. Drafted by representatives with different legal and cultural backgrounds from all regions of the world, the Declaration was proclaimed by the United Nations General Assembly in Paris on 10 December 1948 (General Assembly resolution 217 A) as a common standard of achievements for all peoples and all nations. It sets out, for the first time, fundamental hum


## 3rd step : Embedde and store in a VS
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/u_oJz3v2cSR_lr0YvU6PaA.png" width="50%" alt="split"/>


In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [7]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
docsearch = Chroma.from_documents(texts, embeddings)  # store the embedding in docsearch using Chromadb
print('document ingested')

C:\Users\DELL\AppData\Local\Temp\ipykernel_10232\2099935384.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


document ingested


In [8]:
from langchain_ollama.llms import OllamaLLM


In [9]:
model = OllamaLLM(model="llama3.1:8b", temperature=0.3)

In [10]:
from langchain.chains import RetrievalQA


### Lets try a simple Q&A 

In [11]:
qa = RetrievalQA.from_chain_type(llm=model, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=False)
query = "what is the document about ?"
qa.invoke(query)

{'query': 'what is the document about ?',
 'result': 'The Universal Declaration of Human Rights. \n\nThis document outlines a common standard for human rights that should be respected by all nations, and it sets out various rights and freedoms that individuals are entitled to, including the right to life, liberty, security, education, and cultural participation.'}

In [12]:
query = "can you summarize the document?"
qa.invoke(query)

{'query': 'can you summarize the document?',
 'result': 'The Universal Declaration of Human Rights is a document adopted by the United Nations General Assembly that outlines fundamental human rights and freedoms. It consists of 30 articles that describe the inherent dignity and worth of all individuals, and sets out their basic entitlements to life, liberty, security, education, and other essential aspects of well-being.\n\nThe document emphasizes the importance of promoting respect for these rights and freedoms through education, teaching, and progressive measures at both national and international levels. It also recognizes that everyone is entitled to equal treatment and protection under the law, regardless of their background or circumstances.\n\nSome key themes and principles in the document include:\n\n* The inherent dignity and worth of all individuals\n* Equality and non-discrimination\n* Freedom from slavery, servitude, and arbitrary arrest\n* Right to life, liberty, and secur

### Now lets use a prompt emplate to guide the responses from the llm the way we want

for example if I ask the LLm an irrelevant question

In [13]:
query = "Can I eat in the company?"
qa.invoke(query)

{'query': 'Can I eat in the company?',
 'result': "I don't know. The provided text does not mention anything about eating in a company or workplace setting. It appears to be a document outlining human rights and freedoms as part of the Universal Declaration of Human Rights."}

In [14]:
from langchain.prompts import PromptTemplate


In [20]:
prompt_template="""
# UNIVERSAL DECLARATION OF HUMAN RIGHTS ANALYSIS
## Role: International Human Rights Legal Expert

## DOCUMENT CONTEXT:
{context}

## USER QUERY:
{question}

## ANALYSIS GUIDELINES:
1. **Legal Precision**: Reference specific Articles (e.g., "Article 25 states...")
2. **Historical Context**: Note this was adopted by UN General Assembly in 1948
3. **Interpretation**: Provide legal interpretation when relevant
4. **Limitations**: Clarify what this declaration is and isn't (aspirational vs. legally binding)
5. **Modern Relevance**: Connect to contemporary human rights issues when appropriate

## RESPONSE FORMAT:
- Start with direct answer
- Cite relevant Articles with quotes
- Provide legal context
- Note applicability and limitations
- Keep responses comprehensive but accessible

## IMPORTANT: 
If the query cannot be answered from the UDHR text, respond:
"This specific matter is not directly addressed in the Universal Declaration of Human Rights. The UDHR focuses on fundamental human rights and freedoms"

## EXPERT ANALYSIS:
"""

In [21]:
udhr_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]  
)

In [22]:
udhr_qa = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    chain_type_kwargs={"prompt": udhr_prompt},
    return_source_documents=True,
    verbose=False
)

In [23]:
query = "Can I eat in the company?"
result = udhr_qa.invoke({"query": query}) 
print(result['result'])

**Direct Answer:** There is no explicit mention of eating in the workplace or company premises in the Universal Declaration of Human Rights (UDHR).

**Cited Articles:**
The relevant articles that touch upon aspects related to work, employment, and social protection are:

* Article 23: "Everyone has the right to work... just and favourable conditions of work..."
* Article 25: "Everyone has the right to a standard of living adequate for the health and well-being of himself and of his family..."

**Legal Context:** The UDHR emphasizes the importance of ensuring that individuals have access to basic necessities, including food, clothing, housing, and medical care. However, it does not explicitly address workplace policies or regulations regarding eating in the company.

**Applicability and Limitations:**
The UDHR is an aspirational document, aiming to establish a common understanding of fundamental human rights and freedoms. It serves as a moral and ethical guide for governments and intern

In [24]:
query = "how many articles are there?"
result = udhr_qa.invoke({"query": query}) 
print(result['result'])

**Direct Answer:** There are 30 Articles in the Universal Declaration of Human Rights (UDHR).

**Citing Relevant Articles with Quotes:**

The UDHR is divided into several sections, but for clarity, I will list all 30 Articles:

1. All human beings are born free and equal in dignity and rights.
2. Everyone is entitled to all the rights and freedoms set forth in this Declaration...
3. Everyone has the right to life, liberty and security of person.
4. No one shall be held in slavery or servitude; slavery and the slave trade shall be prohibited in all their forms.
5. Everyone has the right to a fair trial by an impartial tribunal...
6. Everyone is entitled to recognition everywhere as a person before the law.
7. All are equal before the law and are entitled without any discrimination...
8. Everyone has the right to an effective remedy by the competent national tribunals...
9. No one shall be subjected to arbitrary arrest, detention or exile.
10. Everyone is entitled in full equality to a f

### we could even change the prompt template 

In [30]:
prompt_template = """
You are an assistant specialized in the Universal Declaration of Human Rights (UDHR).
Answer the user's question using only the information from the UDHR provided below.

Context (UDHR excerpt):
{context}

Question: {question}

Give a clear and concise answer, mentioning the relevant article if possible.
If the query cannot be answered from the UDHR text, respond:
"This specific matter is not directly addressed in the Universal Declaration of Human Rights. The UDHR focuses on fundamental human rights and freedoms"

"""
udhr_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]  
)
udhr_qa = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    chain_type_kwargs={"prompt": udhr_prompt},
    return_source_documents=True,
    verbose=False
)

### Now we gonna explore how a memory chat can be useful 

In [31]:
query = "What does Article 23 of the UDHR say?"
result = udhr_qa.invoke({"query": query}) 
print(result['result'])

According to Article 23 of the UDHR:

Everyone has the right to work, to free choice of employment, to just and favourable conditions of work and to protection against unemployment.
Everyone, without any discrimination, has the right to equal pay for equal work.
Everyone who works has the right to just and favourable remuneration ensuring for himself and his family an existence worthy of human dignity, and supplemented, if necessary, by other means of social protection.
Everyone has the right to form and to join trade unions for the protection of his interests.


In [33]:
query = "What about Article 25?"
result = udhr_qa.invoke({"query": query}) 
print(result['result'])

According to Article 25, everyone has the right to a standard of living adequate for the health and well-being of themselves and their family, including food, clothing, housing, medical care, and necessary social services. Additionally, they have the right to security in case of unemployment, sickness, disability, widowhood, old age, or other circumstances beyond their control.


here the model cant really remember that I already asked about Article 23 and 25 

In [37]:
query = "How are those two articles related ?"
result = udhr_qa.invoke({"query": query}) 
print(result['result'])

The two articles are related because they both emphasize the importance of respecting the rights and freedoms of others.

Article 2 states that everyone is entitled to all the rights and freedoms without distinction, and Article 29 emphasizes that in exercising their rights and freedoms, individuals shall be subject only to limitations determined by law for the purpose of securing due recognition and respect for the rights and freedoms of others.


the model answering about article 2 & 29 shows clearly the limitation of a normal RetrievalQA

### Lets make the conversation have memory


In [39]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [73]:
memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)

In [74]:
udhr_qa_memory = ConversationalRetrievalChain.from_llm(llm=model, 
                                           chain_type="stuff", 
                                           retriever=docsearch.as_retriever(), 
                                           memory = memory, 
                                           get_chat_history=lambda h : h, 
                                           return_source_documents=False)

In [75]:
#we create a list to save the chat history 
history = []

In [76]:
query = "What does Article 23 of the UDHR say?"
result =udhr_qa_memory.invoke({"question":query}, {"chat_history": history})
print(result["answer"])

According to Article 23 of the Universal Declaration of Human Rights (UDHR), everyone has the following rights:

* The right to work
* The free choice of employment
* Just and favourable conditions of work
* Protection against unemployment
* Equal pay for equal work, without any discrimination
* Just and favourable remuneration that ensures a worthy existence for themselves and their family, supplemented by other means of social protection if necessary
* The right to form and join trade unions for the protection of their interests.


In [77]:
history.append((query, result["answer"]))

In [78]:
query = "What about Article 20?"
result =udhr_qa_memory.invoke({"question":query}, {"chat_history": history})
print(result["answer"])

According to Article 20, everyone has the right to freedom of peaceful assembly and association. No one may be compelled to belong to an association.


In [79]:
history.append((query, result["answer"]))

In [83]:
query = "How are those two articles i asked you about related ?"
result =udhr_qa_memory.invoke({"question":query}, {"chat_history": history})
print(result["answer"])

The two articles are not directly related, but they share a common theme of protecting individual rights and promoting social welfare. Article 20 guarantees freedom of assembly and association, while Article 23 ensures the right to work, fair employment conditions, and protection against unemployment.

However, one could argue that Article 20 is indirectly related to Article 23 in that it allows individuals to form trade unions (Article 23) for the protection of their interests. The right to freedom of assembly and association (Article 20) enables workers to come together and advocate for better working conditions, fair pay, and other benefits enshrined in Article 23.

It's worth noting that these articles are part of the Universal Declaration of Human Rights, which emphasizes the interconnectedness of human rights and freedoms.


In [84]:
history.append((query, result["answer"]))

### wrap everything seen above into an agent 

In [85]:
def qa():
    memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)
    qa = ConversationalRetrievalChain.from_llm(llm=model, 
                                               chain_type="stuff", 
                                               retriever=docsearch.as_retriever(), 
                                               memory = memory, 
                                               get_chat_history=lambda h : h, 
                                               return_source_documents=False)
    history = []
    while True:
        query = input("Question: ")
        
        if query.lower() in ["quit","exit","bye"]:
            print("Answer: Goodbye!")
            break
            
        result = qa({"question": query}, {"chat_history": history})
        
        history.append((query, result["answer"]))
        
        print("Answer: ", result["answer"])

In [ ]:
qa()

Question:  how many article are they 


C:\Users\DELL\AppData\Local\Temp\ipykernel_10232\548420246.py:17: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"question": query}, {"chat_history": history})


Answer:  There are 30 articles in the Universal Declaration of Human Rights.


Question:  can you class them into main categories ? 


Answer:  The text appears to be the Universal Declaration of Human Rights, adopted by the United Nations General Assembly in 1948.

To answer your rephrased question, I can identify several main categories or themes present in the document:

1. **Human Rights and Freedoms**: This category includes articles that define human rights and freedoms, such as the right to life (Article 3), freedom from slavery (Article 4), and protection of moral and material interests (Article 27).
2. **Education and Culture**: Articles 26-28 focus on education, cultural participation, and access to scientific advancement.
3. **Equality and Non-Discrimination**: This category includes articles that emphasize equality before the law (Article 7) and prohibit discrimination based on various factors such as race, color, sex, language, religion, or national origin (Article 2).
4. **Justice and Due Process**: Articles 9-10 deal with fair trial rights, including the right to a public hearing by an independent tribu